<a href="https://colab.research.google.com/github/MangoHaha/MLFromScratch/blob/master/DecisionTree.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [86]:
!pip install sklearn
import numpy as np
from sklearn import datasets
import matplotlib.pyplot as plt
import math
import sys


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import os
os.chdir('/content/drive/My Drive/MLFromScratch')
sys.path.insert(0,"./utils")

from data_manipulation import divide_on_feature
from data_manipulation import train_test_split, standardize
from data_operation import calculate_entropy, accuracy_score
from data_operation import mean_squared_error, calculate_variance
from principal_component_analysis import PCA

Gini impurity and Information Gain Entropy are pretty much the same. And people do use the values interchangeably. Below are the formulae of both:

1.compute the entropy for data-set

2.for every attribute/feature:

       1.calculate entropy for all categorical values
       2.take average information entropy for the current attribute
       3.calculate gain for the current attribute
       
3. pick the highest gain attribute.
4. Repeat until we get the tree we desired.

Gini:Gini(𝐸)=1−∑pi^2

Minimum value of Gini Index will be 0 when all observations belong to one label.
Maximum value of Gini Index could be 0.5 when all target values are equally distributed.
Favors larger partitions.
Uses squared proportion of classes.
Perfectly classified, Gini Index would be zero.
Evenly distributed would be 1 – (1/# Classes).
You want a variable split that has a low Gini Index.
The algorithm works as 1 – ( P(class1)^2 + P(class2)^2 + … + P(classN)^2)


Entropy:𝐻(𝐸)=−∑PilogPi

Information gain: the expected information gain is the change in information entropy Η from a prior state to a state that takes some information as given 

Favors splits with small counts but many unique values.
Weights probability of class by log(base=2) of the class probability
A smaller value of Entropy is better.  That makes the difference between the parent node’s entropy larger.
Information Gain is the Entropy of the parent node minus the entropy of the child nodes.
Entropy is calculated [ P(class1)*log(P(class1),2) + P(class2)*log(P(class2),2) + … + P(classN)*log(P(classN),2)]


In [0]:
class TreeNode():
    def __init__(self, feature_index, split_val, groups, gini):
        self.left=None
        self.right=None
        self.feature_index=feature_index
        self.split_val=split_val 
        self.gini=gini
        self.groups=groups #[left/right gourps]
 

In [0]:
class DecisionTree():
    def __init__(self, max_depth=5, min_size=1):
        self.max_depth = max_depth
        self.min_size  = min_size
        
        

    def fit(self, X, y):
        self.y = y
        self.X = X
        self.num_features = np.shape(X)[1]
        self.class_values = [set(y)]
        self.root = self._get_root_node([i for i in range(len(X))])
        self._build_tree(self.root,1)

    def _get_root_node(self, idxs):
      best_score = sys.maxsize
      best_node = None
      #loop through each entry, and each feature val
      for index in range(self.num_features - 1):
        for idx in idxs:
          groups = self._split_dataset(index, self.X[idx][index], idxs)
          gini = self._calculate_gini(groups, self.class_values)
          if(gini < best_score):
            best_score = gini
            best_node = TreeNode(index, self.X[idx][index], groups, gini)
      return best_node
                         
    #split a dataset based on an attribute and and attribute value
    def _split_dataset(self, index, value, idxs):
      #split list of idxs, into two groups based on value and feature_index
      left = []
      right= []
      for idx in idxs:
        if(self.X[idx][index] < value):
          left.append(idx)
        else:
          right.append(idx)
      return [left, right]
   
    #calcualte geni index for a split dataset
    """
    gini_index = sum(proportion * (1.0 - proportion)) = 1.0 - sum(proportion * proportion)
    
    """
    def _calculate_gini(self, groups, classes):
        #dat: [1,2,3] classes [0,0,1]
        #group 1 [1,2] label: 0; group2 [3] label: 1
        #count all samples at split point
        num_instances = float(sum(len(group) for group in groups))
        gini = 0.0
        #calculate gini for each group, then add weighted val
        for group in groups:
          size = float(len(groups))
          #avoid divided by zero
          if size == 0:
            continue;
          score = 0.0
          #calculate each score based for each class of label
          for label in classes:
            prob = [self.y[idx] for idx in group].count(label)*1.0/size
            score += prob**2.0
          #weight the groups score by size
          gini += (1.0 - score) * (size/num_instances)
        return gini
      
    def _build_tree(self, root, depth=0):
      left, right = root.groups
      #if any either group get all, another get nothing, there is no meaning to 
      #split further, since if you split further, the split critria is still gini
      #index, you will get the same split.
      if(len(left) == 0 or len(right) == 0):
        root.left = root.right = self._terminal(left + right)
        return
      
      if depth >= self.max_depth:
        root.left = self._terminal(left)
        root.right = self._terminal(right)
      
      root.left = self._get_root_node(left)
      self._build_tree(root.left, depth+1)
      root.right = self._get_root_node(right)
      self._build_tree(root.right, depth+1)
      return root
    #create a terminal node value (leaf node stores predicted outcome)
    def _terminal(self, group):
        #print(group)
        outcomes = [self.y[idx] for idx in group]
        #print(outcomes)
        #print(max(set(outcomes), key=outcomes.count))
        return max(set(outcomes), key=outcomes.count)
      
    def predict_one(self, x):
        node = self.root
        while isinstance(node, TreeNode):
            split_index = node.feature_index
            split_value = node.split_val
            if x[split_index] < split_value:
                node = node.left
            else:
                node = node.right
        return node

    def predict(self, X):
        preds = []
        for x in X:
          preds.append(self.predict_one(x))
        #labels = map(self.predict_one, X)
        return preds
     
    def print_tree(self, node, depth=0):
        if isinstance(node, TreeNode):
            print("{} split feature{} at value {} with gini score of {}".format(depth*" ", node.feature_index, node.split_val,node.gini))
            self.print_tree(node.left,depth+1)
            self.print_tree(node.right,depth+1)
        else:
            print("{} {}".format(depth*" ", node))

In [166]:
iris = datasets.load_iris()
X = iris.data[:, :2]
y = (iris.target != 0) * 1
model = DecisionTree()
model.fit(X,y)
preds = model.predict(X)
print(preds)
model.print_tree(model.root)
accuracy = accuracy_score(y.tolist(), preds)
  
print ("Accuracy:", accuracy)


[1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
 split feature0 at value 5.1 with gini score of 0.02666666666666667
  split feature0 at value 4.9 with gini score of 0.125
   split feature0 at value 4.7 with gini score of 0.25
    split feature0 at value 4.6 with gini score of 0.4444444444444444
     split feature0 at value 4.4 with gini score of 0.8
      split feature0 at value 4.3 with gini score of 4.0
       0
       0
      split feature0 at value 4.4 with gini score of 1.0
       0
       0
     split feature0 at value 4.6 with gini score of 1.0
      0
      0
    split feature0 at v